### Environment Setup
Prepare the working environment and ensure that all required dependencies are installed.

**Import libraries**

In [1]:
import cloudscraper
import pandas as pd
from collections import defaultdict

**Ignore future Warnings**

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### Dataset Containers

- **seasons** – a list of all Premier League seasons that will be scraped and processed.  
- **alldata** – a dictionary where each key is a season string and each value is a structure holding all datasets associated with that season.

In [3]:
seasons = [
    "2020-2021",
    "2021-2022",
    "2022-2023",
    "2023-2024",
    "2024-2025",
    "2025-2026"
]

alldata = {}

### Scraping and Merging Methods
Functions and utilities used to retrieve squad and player statistics from FBref and merge them into structured datasets.

**scrape_team_stats**  
A method that returns a specific team stats table scraped from a specific website
- url - the website, where the table is from
- attrs - table id

In [4]:
def scrape_team_stats(url, attrs):
    scraper = cloudscraper.create_scraper()
    html = scraper.get(url).text
    tables = pd.read_html(html, attrs=attrs, header=1)
    df = tables[0]
    df = df.loc[:, ~df.columns.str.startswith("Unnamed")]

    return df

**merge_squad_player_stats_per_season**  
A method that returns a dictionary, where keys are seasons and values are merged datasets.
- data - a dictionary, where keys are seasons and values are all necessary datasets for that season

In [5]:
def merge_squad_player_stats_per_season(data):
    merged_seasons = {}
    for season, df in data.items():
        merged = None
        for name, df in df.items():
            df = df.rename(columns={col: f"{col}" for col in df.columns if col != "Squad"})
        
            if merged is None:
                merged = df
            else:
                merged = merged.merge(df, on="Squad", how="outer")
            merged_seasons[season] = merged

    return merged_seasons

### Possession stats

**clean_and_save_possession_stats**  
A method that returns a cleaned possession dataset with only necessary columns needed for the predictor.
- df - a scraped possession dataset

In [6]:
def clean_and_save_possession_stats(df):
    if all(col in df.columns for col in ["PrgR", "90s"]):
        df["PrgR_per90"] = (df["PrgR"] / df["90s"]).round(2)
    keep_cols = ["Squad", "Poss", "PrgR", "PrgR_per90"]
    df = df[[c for c in keep_cols if c in df.columns]]

    is_against = df["Squad"].astype(str).str.contains("vs ").any()

    if is_against:
        df["Squad"] = df["Squad"].str.replace("vs ", "", regex=False)

    rename_cols = {
        "Poss": "Possession",
        "PrgR_per90": "Progressive_Passes_Received_Per90"
    }
    df = df.rename(columns=rename_cols)

    df = df.drop(columns=["PrgR", "90s"], errors="ignore")

    if is_against:
        df.rename(
            columns={col: col + "_Against" for col in df.columns if col != "Squad"},
            inplace=True
        )

    return df

**Possession for**

Creating *possession for* tables

In [7]:
base_url = "https://fbref.com/en/comps/9/{season}/possession/Premier-League-Stats"
attrs = {"id": "stats_squads_possession_for"}
for season in seasons:
    url = base_url.format(season=season)
    df = scrape_team_stats(url, attrs)
    data = clean_and_save_possession_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["possession_for"] = data

**Possession against**

Creating *possession against* tables

In [8]:
base_url_against = "https://fbref.com/en/comps/9/{season}/possession/Premier-League-Stats"
attrs_against = {"id": "stats_squads_possession_against"}
for season in seasons:
    url = base_url_against.format(season=season)
    df = scrape_team_stats(url, attrs_against)
    data = clean_and_save_possession_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["possession_against"] = data

### Shooting stats

**clean_and_save_shooting_stats**  
A method that returns a cleaned shooting dataset with only necessary columns needed for the predictor.
- df - a scraped shooting dataset

In [9]:
def clean_and_save_shooting_stats(df):
    keep_cols = ["Squad", "Sh/90", "SoT/90"]
    df = df[keep_cols]

    is_against = df["Squad"].astype(str).str.contains("vs ").any()
    if is_against:
        df["Squad"] = df["Squad"].str.replace("vs ", "", regex=False)
        cols = [c for c in df.columns if c != "Squad"]
        df = df.rename(columns={c: c + "_Against" for c in cols})
    rename_cols = {
        "Sh/90": "Shots_Per_90",
        "SoT/90": "Shots_On_Target_Per_90"
    }

    df = df.rename(columns=rename_cols)
    return df

**Shooting for**  
Creating *shooting for* tables

In [10]:
base_url_shoot_for = "https://fbref.com/en/comps/9/{season}/shooting/Premier-League-Stats"
attrs_shoot_for = {"id": "stats_squads_shooting_for"}

for season in seasons:
    url = base_url_shoot_for.format(season=season)
    df = scrape_team_stats(url, attrs_shoot_for)
    data = clean_and_save_shooting_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["shooting_for"] = data

**Shooting against**  
Creating *shooting against* tables

In [11]:
base_url_shoot_against = "https://fbref.com/en/comps/9/{season}/shooting/Premier-League-Stats"
attrs_shoot_against = {"id": "stats_squads_shooting_against"}

for season in seasons:
    url = base_url_shoot_against.format(season=season)
    df = scrape_team_stats(url, attrs_shoot_against)
    data = clean_and_save_shooting_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["shooting_against"] = data

### Attacking stats

**clean_and_save_attacking_stats**  
A method that returns a cleaned attacking dataset with only necessary columns needed for the predictor.
- df - a scraped attacking dataset

In [12]:
def clean_and_save_attacking_stats(df):
    keep_cols = ["Squad", "SCA90", "GCA90"]
    df = df[keep_cols]

    is_against = df["Squad"].astype(str).str.contains("vs ").any()
    if is_against:
        df["Squad"] = df["Squad"].str.replace("vs ", "", regex=False)
        cols = [c for c in df.columns if c != "Squad"]
        df = df.rename(columns={c: c + "_Against" for c in cols})
    rename_cols = {
        "SCA90": "Shot_Creating_Actions_Per90",
        "GCA90": "Goal_Creating_Actions_Per90"
    }

    df = df.rename(columns=rename_cols)
    return df

**Attacking for**  
Creating an *attacking for* table

In [ ]:
base_url_att_for = "https://fbref.com/en/comps/9/{season}/gca/Premier-League-Stats"
attrs_attacking_for = {"id": "stats_squads_gca_for"}

for season in seasons:
    url = base_url_att_for.format(season=season)
    df = scrape_team_stats(url, attrs_attacking_for)
    data = clean_and_save_attacking_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["attacking_for"] = data

**Attacking against**  
Creating an *attacking against* table.

In [ ]:
base_url_att_against = "https://fbref.com/en/comps/9/{season}/gca/Premier-League-Stats"
attrs_attacking_against = {"id": "stats_squads_gca_against"}

for season in seasons:
    url = base_url_att_against.format(season=season)
    df = scrape_team_stats(url, attrs_attacking_against)
    data = clean_and_save_attacking_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["attacking_against"] = data

### Importing tables

**Merging datasets into a complete dataset**  
Combine all season-specific CSV files into a unified dataset that includes possession, shooting, and attacking statistics for each team. Ensure that the joins use consistent team names and matching keys across all files.

**Saving the datasets into the correct folders**  
Store the merged datasets in the appropriate season folders under `CSV_files/`, following the project’s directory structure. Each processed file should overwrite or extend the existing season-level data to keep the repository organized and ready for model training.

In [ ]:
merge = merge_squad_player_stats_per_season(alldata)
for season, df in merge.items():
    path = f"../CSV_files/Season_{season}/squad_player_stats.csv"
    df.to_csv(path, index=False)